Before we get started: 
1) install pydeck if you don't have it in your environment already

In [ ]:
# !pip install pydeck

# Learning goals
After this week's lesson you should be able to:
- Style a map in geopandas
- Add multiple layers onto a map
- Create a map with a basemap using `.explore()`
- Create an interactive map using geopandas and pydeck


This week's lessons are adapted from:
- [The Geopandas user guide](https://geopandas.org/en/stable/docs/user_guide.html)
- [Pydeck documentation](https://pypi.org/project/pydeck/)

In [ ]:
# We are going to start importing the libraries we need
# all in one cell. 
# It is a good practice to keep all the imports in one cell so that
# we can easily see what libraries we are using in the notebook.

import pandas as pd
import geopandas as gpd

### This is a new library we have not used before
### Matplotlib is a charting library. 
### We will talk about it properly on Thursday.
import matplotlib.pyplot as plt

%matplotlib inline
# we use the inline backend to generate the plots within the browser


import pydeck as pdk


## 0. Read in the data
We are going to map the 2020 population density in NYC at the block group level. I've created a `GeoJSON` called `blkgrp_acs.geojson` from the [NHGIS](data2.nhgis.org) data portal. You can also download it [here](https://www.dropbox.com/s/p6vtkmvjzfrt66p/blkgrp_acs.geojson?dl=0).

In [ ]:
blkgrp_acs = gpd.read_file('blkgrp_acs.geojson')

Note here that we have a column named `pop_density`. We'll just be using this in this example.

In [ ]:
blkgrp_acs.head()

# 1. Static Maps in Geopandas 
This should mostly feel like a refresher as we've indirectly covered some of these tools. I will present it more systematically today. 

## 1.1 Plotting
When working with a geodataframe in geopandas, we have a function called `.plot()` that is using a library called `matplotlib` under the hood. 

In [ ]:
blkgrp_acs.plot()

#### Small Detour: Reading Documentation
Here are all the possible parameters that we can include in the `.plot()` function. The following cell is a snapshot from the [geopandas documentation](https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoDataFrame.plot.html) that tell us a bit more about what the function does. 

Let's take a look at the first one: 
- This tells us that we can specify a parameter called `column`, that takes as input a `str` type. 
- What does this mean? This means that if we expand our code to be something like `gdf.plot(column='my_col_name')`, the values from the `my_col_name` column in my geodataframe will be used to color the plot. 
- In our example, since we are interested in `pop_density`, we might want to plot this, like so: 
    - `blkgrp_acs.plot(column='pop_density')`
- You may also have seen this version: `blkgrp_acs.plot('pop_density')`
    - This is a shorthand way to write `blkgrp_acs.plot(column='pop_density')`. Python knows that, when we don't specify the parameter, the first input in a function corresponds to the first parameter, the second input to the second paramter, and so on.
- We can also see that, by default, this input is `None`, meaning there is no value in `column` by default.

</figure>
<img src="https://www.dropbox.com/s/z6nezt2u3payzmj/Screen%20Shot%202023-02-11%20at%201.14.07%20PM.png?dl=1" alt="drawing" width="1000" style="display: block; margin: 0 auto"/>
</figure>


Let's move on to the fourth input - **color**. The documentation stays that this takes a `str` as an input and 
>"If specified, all objects will be colored uniformly."

In [ ]:
blkgrp_acs.plot(color='red')

Since we know we want to look at the column `pop_density` let's go ahead and set our `column` input to be `pop_density`.

In [ ]:
blkgrp_acs.plot(column='pop_density')
# This is not very informative because the default color scheme give us no idea of the range of values.

## 1.2 Changing the figure size of a plot
We can also see that there is an input called **figsize**. The documentation says: 
- This takes as an input a "tuple of integers", which is  `(number1,number2)`. Remember that a tuple is anything separated by parentheses, typically only with two elements (hence the word). 
- What this controls is "Size of the resulting matplotlib.figure.Figure. If the argument axes is given explicitly, figsize is ignored.".


In [ ]:
# At least now we can see those tiny block groups a little better.. 
# There is some variation in the population density, but it is still hard to tell what the range is.
blkgrp_acs.plot(column='pop_density',
                figsize=(12,12))

## 1.3 Adding a legend
To help us understand what values these colors are mapping to, let's introduce a legend. 

In [ ]:
## The legend automatically shows the range of values in the data.
blkgrp_acs.plot(column='pop_density',
                figsize=(12,12),
                legend=True)

## 1.4 Schemes 
By default geopandas uses an equal interval classification scheme. This means that the range of values (where range is the max_val - min_val) is divided equally. The scheme above has the colors continuous along this interval. (Typically, we use a classification scheme based on **k** distinct categories.)

Notice that we have the input option **scheme**. The documentation says that this takes a `str` input type, which is
>"Name of a choropleth classification scheme (requires mapclassify). A mapclassify.MapClassifier object will be used under the hood. Supported are all schemes provided by mapclassify (e.g. ‘BoxPlot’, ‘EqualInterval’, ‘FisherJenks’, ‘FisherJenksSampled’, ‘HeadTailBreaks’, ‘JenksCaspall’, ‘JenksCaspallForced’, ‘JenksCaspallSampled’, ‘MaxP’, ‘MaximumBreaks’, ‘NaturalBreaks’, ‘Quantiles’, ‘Percentiles’, ‘StdMean’, ‘UserDefined’). Arguments can be passed in classification_kwds."

### 1.4.1 Quantiles
Instead of the default equal interval scheme, let's try a quantiles scheme `Quantiles`. This will separate my data into 5 parts with the same number of values so that 20% of the data is below the first quantile, 20% is between the first and second quantile, and so on.

Now we have more variation in the colors. 

In [ ]:
## Notice that the legend now gives us a range of values for each color.
## The values have been rounded to the nearest two significant digits.
## That is why some of the ranges are the same.
blkgrp_acs.plot(column='pop_density',
                figsize=(12,12),
                legend=True,
                scheme= 'Quantiles')

### 1.4.2 Natural breaks
Another common classification scheme is **natural breaks**, which is (from the [ESRI GIS dictionary](https://support.esri.com/en-us/gis-dictionary/natural-breaks-classification#:~:text=%5Bcartography%5D%20A%20method%20of%20manual,the%20low%20points%20of%20valleys.)): 
>A method of manual data classification that seeks to partition data into classes based on natural groups in the data distribution. Natural breaks occur in the histogram at the low points of valleys. Breaks are assigned in the order of the size of the valleys, with the largest valley being assigned the first natural break.

In [ ]:
blkgrp_acs.plot(column='pop_density',
                figsize=(12,12),
                legend=True,
                scheme= 'NaturalBreaks')

### 1.4.3 Boxplot
Can anyone guess what a boxplot scheme means? 

In [ ]:
blkgrp_acs.plot(column='pop_density',
                figsize=(12,12),
                legend=True,
                scheme= 'BoxPlot')

## 1.5 Legend `legend_kws` 

Notice that we have two types of legends: 
- **Binned**
- **Continuous**

The following are some legend tools but some only apply to either binned or continuous legends. 

Because geopandas is using matplotlib under the hood, but does not expose all the optionalities of matplotlib, you will often see optional inputs like `something_kws` (kws = keywords) that allow us to access those features with this. 

Here, let's try using `legend_kws`, which takes in a `dict` input format with 
>Keyword arguments to pass to matplotlib.pyplot.legend() or matplotlib.pyplot.colorbar(). 

Notice here that `legend_kws`. You can read the `matplotlib.pyplot.legend()` [documentation](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.legend.html#matplotlib.pyplot.legend) to get a better sense of all the possible ways to format the legend, but let's go over a few keys ones: 

### 1.5.1 Location of Legend (binned): 


In [ ]:
blkgrp_acs.plot(column='pop_density',
                figsize=(12,12),
                legend=True,
                scheme= 'BoxPlot',
                legend_kwds={'loc': 'lower left'})

In [ ]:
blkgrp_acs.plot(column='pop_density',
                figsize=(12,12),
                legend=True,
                scheme= 'BoxPlot',
                legend_kwds={'loc': 'upper left'})

### 1.5.2 Orientation (continuous)
For continuous legends, we can't change the location of the legend, but we can use the key, `orientation`. 

In [ ]:
blkgrp_acs.plot(column='pop_density',
                figsize=(12,12),
                legend=True,
                legend_kwds={'orientation':'horizontal'})

### 1.5.3 Sizing a legend (continuous)
If you are as bothered by the size the continuous legend as I am, here are to code snippets for fixing that. (I'm not going to explain what they're doing here. )

Vertical legend

In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable

fig, ax = plt.subplots(1, 1,figsize=(12,12))
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.1)

blkgrp_acs.plot(column='pop_density',
                ax=ax,
                cax=cax,
                legend=True,
                )

## 1.6 Colors

From the [matplotlib documentation](https://matplotlib.org/stable/gallery/color/colormap_reference.html), here is are some of the named color schemes: 


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

cmaps = [('Perceptually Uniform Sequential', [
            'viridis', 'plasma', 'inferno', 'magma', 'cividis']),
         ('Sequential', [
            'Greys', 'Purples', 'Blues', 'Greens', 'Oranges', 'Reds',
            'YlOrBr', 'YlOrRd', 'OrRd', 'PuRd', 'RdPu', 'BuPu',
            'GnBu', 'PuBu', 'YlGnBu', 'PuBuGn', 'BuGn', 'YlGn']),
         ('Sequential (2)', [
            'binary', 'gist_yarg', 'gist_gray', 'gray', 'bone', 'pink',
            'spring', 'summer', 'autumn', 'winter', 'cool', 'Wistia',
            'hot', 'afmhot', 'gist_heat', 'copper']),
         ('Diverging', [
            'PiYG', 'PRGn', 'BrBG', 'PuOr', 'RdGy', 'RdBu',
            'RdYlBu', 'RdYlGn', 'Spectral', 'coolwarm', 'bwr', 'seismic']),
         ('Cyclic', ['twilight', 'twilight_shifted', 'hsv']),
         ('Qualitative', [
            'Pastel1', 'Pastel2', 'Paired', 'Accent',
            'Dark2', 'Set1', 'Set2', 'Set3',
            'tab10', 'tab20', 'tab20b', 'tab20c']),
         ('Miscellaneous', [
            'flag', 'prism', 'ocean', 'gist_earth', 'terrain', 'gist_stern',
            'gnuplot', 'gnuplot2', 'CMRmap', 'cubehelix', 'brg',
            'gist_rainbow', 'rainbow', 'jet', 'turbo', 'nipy_spectral',
            'gist_ncar'])]

gradient = np.linspace(0, 1, 256)
gradient = np.vstack((gradient, gradient))


def plot_color_gradients(cmap_category, cmap_list):
    # Create figure and adjust figure height to number of colormaps
    nrows = len(cmap_list)
    figh = 0.35 + 0.15 + (nrows + (nrows-1)*0.1)*0.22
    fig, axs = plt.subplots(nrows=nrows, figsize=(6.4, figh))
    fig.subplots_adjust(top=1-.35/figh, bottom=.15/figh, left=0.2, right=0.99)

    axs[0].set_title(f"{cmap_category} colormaps", fontsize=14)

    for ax, cmap_name in zip(axs, cmap_list):
        ax.imshow(gradient, aspect='auto', cmap=cmap_name)
        ax.text(-.01, .5, cmap_name, va='center', ha='right', fontsize=10,
                transform=ax.transAxes)

    # Turn off *all* ticks & spines, not just the ones with colormaps.
    for ax in axs:
        ax.set_axis_off()



In [ ]:

for cmap_category, cmap_list in cmaps:
    plot_color_gradients(cmap_category, cmap_list)

In [ ]:
blkgrp_acs.plot(column='pop_density',
                figsize=(12,12),
                legend=True,
                scheme= 'Quantiles',
                cmap='cool')

## Q.1 
Would you use a diverging colormap to make a choropleth for population density? Why or why not? Please answer in markdown text in the new cell below this one. 

INSERT YOUR TEXT HERE

# 2. Maps with multiple layers

Use your parks properties data from last week 

`Parks Properties_20240130.zip`


In [ ]:
parks = gpd.read_file('Parks Properties_20240130.zip')

We are going to plot parks on top of the Census Block Group data. Let's make sure they are the same projection first

In [ ]:
parks.crs

In [ ]:
blkgrp_acs.crs

## 2.2 Method 1
The first method to map multiple layers is to assign your plot to a variable, and then then assign the subsequent layers' `ax` to this. 

In [ ]:
## Step 1
base = blkgrp_acs.plot(column='pop_density',
                figsize=(12,12),
                legend=True,
                scheme= 'Quantiles',
                cmap='Blues')


parks.plot(ax = base,color='green')

Note: If you try to plot multiple legends, without more complex manipulations, the plot will only show the legend from the last layer to be plotted. 

In [ ]:
base = blkgrp_acs.plot(column='pop_density',
                figsize=(12,12),
                legend=True,
                scheme= 'Quantiles',
                cmap='Blues')
parks.plot(ax = base,column='typecatego',cmap= 'tab20',legend=True)

## 2.3 Method 2
The second, and generally more preferred, method is to use a `fig,ax = plt.subplots()` construction. 

Briefly, here is what a figure is in relation to axes: 

<img src="https://www.earthdatascience.org/images/earth-analytics/plot-data/fig-2-plots.png" alt="drawing" width="60%"/>


In [ ]:
## The values in subplots are (numer of rows, numer of columns,figsize)
fig1,ax1 = plt.subplots(1,1,figsize=(12,12))

## Here we are only creating on axis so we can plot the two layers on the same axis.
blkgrp_acs.plot(column='pop_density',
                ax=ax1,
                legend=True,
                scheme= 'Quantiles',
                cmap='Blues')

parks.plot(ax = ax1,column='typecatego',legend=True)

## 2.4 Plotting in a grid
The values in subplots are (number of rows, number of columns,figsize). So `plt.subplots(x,y)` will create a figure of x number of rows and y number of columns. 

As an example, here I've made a figure (`fig`) with 4 rows and 5 columns of empty axes (`ax`)

In [ ]:
fig,ax = plt.subplots(4,5,figsize = (12,12))

Notice that `ax` is an array of subplots.

In [ ]:
ax

In [ ]:
ax.shape

In [ ]:
## This allows me to access the second row and second column axes object. 
ax[1,1]

We can then assign a chart to each axes object

In [ ]:
## The values in subplots are (numer of rows, numer of columns,figsize)
fig1,ax1 = plt.subplots(1,2,figsize=(10,5))

In [ ]:
ax1

In [ ]:
## The values in subplots are (numer of rows, numer of columns,figsize)
fig1,ax1 = plt.subplots(1,2,figsize=(10,5))

blkgrp_acs.plot(column='pop_density',
                ax=ax1[0],
                scheme= 'Quantiles',
                cmap='Blues')

parks.plot(ax = ax1[1],column='typecatego')

# 3. Interactive maps with `.explore()` 
As we've seen, geopandas has a function called `explore()` that allows us to make a quick interactive map within the jupyter notebook. This tool is built with using some geopandas functionality paired with a library called `folium`, which is itself a wrapper for a front-end javascript library used to make interactive maps for a browser called `leaflet.js`. 

As you can see, part of the benefit of having this this map is that we also get a basemap. 

In [ ]:
blkgrp_acs.explore()

## 3.1 Basemaps

There a few other built-in basemap tiles you can use: 

`["OpenStreetMap", "CartoDB positron", “CartoDB dark_matter"]`

In [ ]:
blkgrp_acs.explore(tiles='CartoDB positron')

## 3.2 Styling
Styling works very similarly as plot. You can indicate: 
- which column you'd like to style by using `column`
- What binning scheme using `scheme`
- What colors using `cmap`
- Whether you want a legend using `legend`


You can also indicate other features like: 
- If you want tooltips using `tooltip=False` or `tooltip=True`
- What pop-ups (on click) you might want show using `popup=["Col1","Col2",...]`

In [ ]:
blkgrp_acs.explore(tiles='CartoDB dark_matter',
                   column='pop_density',
                   scheme='Quantiles',
                   cmap='viridis',
                   tooltip=False,
                   popup=['pop_density'])

## 3.3 Plotting multiple layers

In [ ]:
import folium
nyc_map = blkgrp_acs.explore(tiles='CartoDB positron',
                column='pop_density',
                scheme='Quantiles',
                cmap='viridis',
                legend=True,  # show legend
                k=10,  # use 10 bins
                tooltip=False,
                popup=['pop_density'],
                style_kwds={
                'stroke':False}, # changes the styling of the layer. 
                name="blkgrp",  # name of the layer in the map
                   )

parks.explore(
    m=nyc_map,  # pass the map object
    color="green",  # use red color on all points
    tooltip="typecatego",  # show "name" column in the tooltip
    tooltip_kwds=dict(labels=False),  # do not show column label in the tooltip
    style_kwds={'fillOpacity': 0.7,
                'stroke':False}, # changes the styling of the layer. 
    name="parks",  # name of the layer in the map
)

# folium.TileLayer("CartoDB positron", show=False).add_to(
#     nyc_map
# )  # use folium to add alternative tiles
folium.LayerControl().add_to(nyc_map)  # use folium to add layer control

nyc_map # show map

# 4. Pydeck
pydeck is tool for rendering data in python, especially for larger datasets. It takes advantage of a tool called `deck.gl` for making 2D and 3D graphics in your browser by taking advantage of the graphics card (GPU) on your computer. 

There are two basic components that we'll be using: 
- layers
- a deck

The `pdk.Layer` object specifies the data and styling for each layer we want to visualize.

The `pdk.Deck` object is what renders our layers. 

## 4.1 Creating a basic map

In [ ]:
layers = [
    pdk.Layer(
        "GeoJsonLayer",
        data=blkgrp_acs,
        opacity=1,
        stroked=False,
        filled=True,
        extruded=True,
        wireframe=False,
        pickable=True,
        get_elevation="pop_density", ## I want the height of the extrusion to be the population density
        get_fill_color=[180, 0, 200, 140]
        
    )
]
r = pdk.Deck(layers=layers)

## We have to save the map to a file to see it.
## Note that you can also open this file in a web browser.
r.to_html("pydeck_example.html")


It doesn't look like we have anything mapped. However, zoom into NYC and you should see your data. 

We can see the default is a 2D map and, therefore, we can't see the extrusions. 


**We need to set 'initial view state' so that our map has a pitch, zoom, and starting lat/lng for our extrusions to appear.**

In [ ]:

INITIAL_VIEW_STATE = pdk.ViewState(latitude=40.730610, 
                                   longitude= -73.935242, 
                                   zoom=9, max_zoom=16, pitch=60, bearing=0)
layers = [
    pdk.Layer(
        "GeoJsonLayer",
       
        data=blkgrp_acs,
        opacity=1,
        stroked=False,
        filled=True,
        extruded=True,
        wireframe=False,
        pickable=True,
        get_elevation="pop_density",
        get_fill_color=[180, 0, 200, 140] ##  Set an RGBA value for fill
        
    )
]

## Note that we are passing the initial view state as a parameter to the Deck object.
r = pdk.Deck(layers=[layers] ,  map_style='light',initial_view_state=INITIAL_VIEW_STATE)
r.to_html("pydeck_example.html")


Now we at least have the viewport in the right location, pitch, and zoom. However, we still can't see the extrusions. 

That's because we are trying to visalize the pop. density, which is small!

In [ ]:
blkgrp_acs['pop_density'].describe()

We can write a formula right in our layers specification to transform the pop. density into a bigger number by multiplying it by 30,000.

In [ ]:

INITIAL_VIEW_STATE = pdk.ViewState(latitude=40.730610, 
                                   longitude= -73.935242, 
                                   zoom=9, max_zoom=16, pitch=60, bearing=0)
layers = [
    pdk.Layer(
        "GeoJsonLayer",
        data=blkgrp_acs,
        opacity=1,
        stroked=False,
        filled=True,
        extruded=True,
        wireframe=False,
        pickable=True,
        get_elevation="pop_density *30000",
        get_fill_color=[180, 0, 200, 140]
        
    )
]

## Note that we are passing the initial view state as a parameter to the Deck object.
r = pdk.Deck(layers=[layers] ,  map_style='light',initial_view_state=INITIAL_VIEW_STATE)
r.to_html("pydeck_example.html")


We can also vary the color by the population density.

In [ ]:

INITIAL_VIEW_STATE = pdk.ViewState(latitude=40.730610, 
                                   longitude= -73.935242, 
                                   zoom=9, max_zoom=16, pitch=60, bearing=0)
layers = [
    pdk.Layer(
        "GeoJsonLayer",
        data=blkgrp_acs,
        opacity=1,
        stroked=False,
        filled=True,
        extruded=True,
        wireframe=False,
        pickable=True,
        get_elevation="pop_density *30000",
        get_fill_color='[0, 255, pop_density * 10, 255 ]' ## The four values are the RGBA values for fill: [red, green, blue, alpha/opaqueness] from 0-255.
        
    )
]

## Note that we are passing the initial view state as a parameter to the Deck object.
r = pdk.Deck(layers=[layers] , initial_view_state=INITIAL_VIEW_STATE)
r.to_html("pydeck_example.html")


Why didn't anything change? 


In [ ]:
(blkgrp_acs['pop_density']*10).describe()

I still have very small numbers. So I essentially have [0,255,0-ish,255] which gets me color that is essentially only green

Here, I'm going to use a function that will allow me to map my population density values onto a set of numbers from 0 to 255. 

In [ ]:
from scipy.interpolate import interp1d
m = interp1d([blkgrp_acs['pop_density'].min(), blkgrp_acs['pop_density'].max()], [0, 255])  

blkgrp_acs['pop_density_color'] = blkgrp_acs['pop_density'].apply(lambda x: m(x))

In [ ]:

INITIAL_VIEW_STATE = pdk.ViewState(latitude=40.730610, 
                                   longitude= -73.935242, 
                                   zoom=9, max_zoom=16, pitch=60, bearing=0)
layers = [
    pdk.Layer(
        "GeoJsonLayer",
        data=blkgrp_acs,
        opacity=1,
        stroked=False,
        filled=True,
        extruded=True,
        wireframe=False,
        pickable=True,
        get_elevation="pop_density *30000",
        get_fill_color='[0, 255, pop_density_color, 255 ]' ## The four values are the RGBA values for fill: [red, green, blue, alpha/opaqueness] from 0-255.
        
    )
]

## Note that we are passing the initial view state as a parameter to the Deck object.
r = pdk.Deck(layers=[layers] , initial_view_state=INITIAL_VIEW_STATE)
r.to_html("pydeck_example.html")


Still not great, but slightly better. 

# Q.2 Create a population density map in pydeck
Create a map in pydeck where you visualize the population density (or some transformation of it) using: 

- Extrusions 
- A choropleth map (in which pop. density is mapped onto a color scheme)